# Create the files needed for the preproc of PPMI dataset
## 1. Subject-session csv files
## 2. tbd

In [1]:
# libs and envs
import sys
from pathlib import Path
import pandas as pd
import numpy as np

#proj_dir='/data/pd/ppmi' # BIC
proj_dir='/scratch' # CC
codes_dir_str=proj_dir+'/mr_proc'
sys.path.append(codes_dir_str)

# main PATH
codes_dir    = Path(codes_dir_str)
bids_dir_str = proj_dir+'/runningMDD_BIDS' # PPMI_BIDS
bids_dir     = Path(bids_dir_str)
fmriprep_dir = codes_dir / 'fMRIPrep' 

# output subject session list 
col_names = ['subject', 'session']
ppmi_subj_ses_file   = fmriprep_dir / 'ppmi_subject_session.csv'  # Information from download database.

In [5]:
# read bids layout and output all sessions
from bids import BIDSLayout
ppmi_layout=BIDSLayout(bids_dir)
print(ppmi_layout.get_sessions())

/usr/local/miniconda/lib/python3.8/site-packages/bids/layout/models.py:148: FutureWarning: The 'extension' entity currently excludes the leading dot ('.'). As of version 0.14.0, it will include the leading dot. To suppress this warning and include the leading dot, use `bids.config.set_option('extension_initial_dot', True)`.
  warnings.warn("The 'extension' entity currently excludes the leading dot ('.'). "


['1', '2', '3']

In [6]:
# select all the available T1w images and create the subject session list 
save_file=0

suffix    = 'T1w'
extension = 'nii.gz'
ppmi_file_list=ppmi_layout.get(suffix=suffix, extension=extension, return_type='file')
ppmi_file_names=[x.split('/')[-1] for x in ppmi_file_list]

subj_ses_df = pd.DataFrame({col_names[0]:[x.split('_')[0] for x in ppmi_file_names], col_names[1]: [x.split('_')[1].split('-')[-1] for x in ppmi_file_names]})
print(subj_ses_df)

# Generate subject,session file for fMRIPrep preporocessing 
if save_file:
    subj_ses_df.to_csv(ppmi_subj_ses_file, header=False, index=False)

           subject session
0    sub-season101       1
1    sub-season101       2
2    sub-season101       3
3    sub-season102       1
4    sub-season102       2
..             ...     ...
139  sub-season216       2
140  sub-season216       3
141  sub-season217       1
142  sub-season217       2
143  sub-season217       3

[144 rows x 2 columns]


In [3]:
# Read and check the existing subject session list 

subj_ses_exist_df = pd.read_csv(ppmi_subj_ses_file, sep=',', header=None, index_col=None, names= col_names)
subj_ses_exist_df

,subject,session
0,sub-01218AUG16,0
1,sub-3101,1
2,sub-3101,7
3,sub-3102,1
4,sub-3102,7
...,...,...
1452,sub-103813,1
1453,sub-105711,1
1454,sub-112729,1
1455,sub-113369,1


In [5]:
sed -n '/sub/s/^.*,\([0-9]*\).*$/\1/p' /scr | sort -u

SyntaxError: invalid syntax (3543981368.py, line 1)